In [3]:
print("OK !")

OK !


In [4]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
# LangChain wrapper for Pinecone
from langchain_community.vectorstores import Pinecone
# Official Pinecone client
from pinecone import Pinecone as PineconeClient

from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [5]:
PINECONE_API_KEY = "pcsk_4Fh75S_MG7KLkX6D67K5h4GgMBe6mikgyfQVg9yU1FyYsqisZG3rsyveHPSNSArtgSL6na"
PINECONE_API_ENV = "gcp-starter"

In [8]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [9]:
extracted_data = load_pdf("../data/")

In [10]:

#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [11]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5859


In [12]:
!pip install sentence-transformers langchain

try:
    from langchain_community.embeddings import HuggingFaceEmbeddings
except ImportError:
    from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# test
vector = embeddings.embed_query("hello world")
print("Vector length:", len(vector))


c:\Users\indra\anaconda3\envs\mchatbot\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\indra\anaconda3\envs\mchatbot\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Vector length: 384


In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_community.vectorstores import Pinecone as PineconeStore

# --- 1. Set API Key ---
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

# --- 2. Init Pinecone client ---
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

# --- 3. Create index if not exists ---
index_name = "m-chat"

if index_name not in [i["name"] for i in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=len(embeddings.embed_query("hello world")),  # e.g. 384 for MiniLM-L6-v2
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")   # change region if needed
    )

# --- 4. Get handle to index ---
index = pc.Index(index_name)

# --- 5. Insert your chunks into Pinecone via LangChain wrapper ---
docsearch = PineconeStore.from_texts(
    [t.page_content for t in text_chunks],
    embeddings,
    index_name=index_name,
    namespace="medical-book"
)

print("✅ Data inserted successfully into Pinecone!")


✅ Data inserted successfully into Pinecone!


In [34]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index("medical-chatbot", embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result []


In [21]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [22]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [40]:
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

ImportError: Could not import `ctransformers` package. Please install it with `pip install ctransformers`

In [ ]:
from langchain_community.vectorstores import Pinecone
from langchain.chains import RetrievalQA
retriever = docsearch.as_retriever(search_kwargs={"k": 2})

# build RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\indra\anaconda3\envs\mchatbot\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\indra\AppData\Local\Temp\ipykernel_16184\439520835.py", line 6, in <module>
    qa = RetrievalQA.from_chain_type(
  File "c:\Users\indra\anaconda3\envs\mchatbot\lib\site-packages\langchain\chains\retrieval_qa\base.py", line 104, in from_chain_type
    @classmethod
  File "c:\Users\indra\anaconda3\envs\mchatbot\lib\site-packages\langchain\load\serializable.py", line 74, in __init__
  File "pydantic\\main.py", line 347, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 1 validation error for RetrievalQA
retriever
  Can't instantiate abstract class BaseRetriever with abstract methods _get_relevant_documents (type=type_error)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

NameError: name 'qa' is not defined